In [35]:
import numpy as np
from aocd.models import Puzzle


def create_two_d_matrix(input_data):
    data = input_data.split("\n")
    for i in range(0, len(data)):
        data[i] = list(data[i])
    return np.array(data)

# Tips

## Define the object
- `puzzle = Puzzle(year=2017, day=20)` 
- `Puzzle(2017, 20) at 0x107322978 - Particle Swarm>`
- get the input `puzzle.input_data`
- submit by setting:
  - `puzzle.answer_a = value_a`
  - `puzzle.answer_b = value_b`

## Transform to list variables on multiple lines: 
 - t = '''asd
        asd 
        asd 
        asd'''.split('\n')

## Map a string list to integer 
- `map(int, list)`
- `np.array(p.input_data.split('\n'), dtype='int')`

## Day 1

In [36]:
puzzle = Puzzle(year=2024, day=1)


def get_distance(a: np.array, b: np.array) -> int:
    a.sort()
    b.sort()
    return np.sum(np.abs(a - b))


def get_similarity(a: np.array, b: np.array) -> int:
    sim = 0
    for el in a:
        sim += el * np.sum(b == el)
    return sim


# tests
data = puzzle.examples[0].input_data.split("\n")
data = np.array([list(map(int, line.split())) for line in data])
assert get_distance(data[:, 0], data[:, 1]) == 11
assert get_similarity(data[:, 0], data[:, 1]) == 31

# part a
data = puzzle.input_data.split("\n")
data = np.array([list(map(int, line.split())) for line in data])
puzzle.answer_a = get_distance(data[:, 0], data[:, 1])

# part b
puzzle.answer_b = get_similarity(data[:, 0], data[:, 1])

coerced int64 value np.int64(1580061) for 2024/01 to '1580061'
coerced int64 value np.int64(23046913) for 2024/01 to '23046913'


# Day 2

In [37]:
puzzle = Puzzle(year=2024, day=2)

test_data = [
    list(map(int, l.split())) for l in puzzle.examples[0].input_data.split("\n")
]
data = [list(map(int, l.split())) for l in puzzle.input_data.split("\n")]

In [38]:
def possible_bad_level(inc: np.array, dec: np.array, gaps: np.array):
    """Find the indices that could be removed to make the list safe.

    Args:
        inc (np.array): boolean where the list is increasing
        dec (np.array): boolean where the list is decreasing
        gaps (np.array): boolean where the gap are within the [0, 3] limits

    Returns:
        list: Possible bad levels to try
    """

    inc_ix = np.where(~inc)[0]
    dec_ix = np.where(~dec)[0]
    gaps_ix = np.where(~gaps)[0]

    # we add the index where it occurs and the next
    # 9 8 7 4 5 -> increase at index 3 and must remove index 3
    # 9 8 7 6 7 -> increase at index 3 but must remove 4
    # same ideas is valid for the other conditions
    bad_level = set()

    # can't fix more than 2 issues for a condition
    # there is for sure a way to optimize this better
    if len(inc_ix) == 1:
        bad_level.update([inc_ix[0], inc_ix[0] + 1])

    if len(dec_ix) == 1:
        bad_level.update([dec_ix[0], dec_ix[0] + 1])

    if len(gaps_ix) and len(gaps_ix) <= 2:
        bad_level.update([gaps_ix[0], gaps_ix[0] + 1])

    return bad_level


def safe_list(data: np.array, part_b=False) -> bool:
    """Check if the list is safe based on monotonicity and gap jump.

    Args:
        data (np.array): input list to validate
        part_b (bool, optional): Also attempt to validate by removing a single element.
            Defaults to False.

    Returns:
        bool: True if the list is safe otherwise False
    """
    diff = np.diff(data)
    inc = diff > 0
    dec = diff < 0
    safe_gap = (np.abs(diff) >= 0) & (np.abs(diff) <= 3)

    if not (all(inc) or all(dec)) or not all(safe_gap):
        if part_b and (indices := possible_bad_level(inc, dec, safe_gap)):
            return any(safe_list(np.delete(data, ix)) for ix in indices)
        return False

    return True


# test cases
assert sum([safe_list(d) for d in test_data]) == 2
assert sum([safe_list(d, part_b=True) for d in test_data]) == 4

puzzle.answer_a = sum([safe_list(d) for d in data])
puzzle.answer_b = sum([safe_list(d, part_b=True) for d in data])

# Day 3

In [39]:
puzzle = Puzzle(year=2024, day=3)

example_data = puzzle.examples[0].input_data
example_data_b = (
    "xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))"
)
data = puzzle.input_data

In [40]:
def eval_mul(data: str, part_b: bool = False) -> int:
    """From the input string, return the product of all X,Y values
    between the mul(X,Y) pattern.

    Args:
        data (str): input data
        part_b (bool, optional): Look for don't() and do() to stop processing.
            Defaults to False.

    Returns:
        int: sum of the product of all X,Y values identified
    """
    mul = 0
    process = True

    for i in range(0, len(data)):
        next_op = data[i : i + 4]
        possible_stop = data[i : i + 7]

        if part_b:
            if possible_stop == "don't()":
                process = False
            elif next_op == "do()":
                process = True

        if process and next_op == "mul(":
            lhs = data[i + 4 :].split(",")[0]
            rhs = data[i + 4 + len(lhs) + 1 :].split(")")[0]

            if lhs.isdigit() and rhs.isdigit() and " " not in lhs and " " not in rhs:
                mul += int(lhs) * int(rhs)
    return mul


# tests
assert eval_mul(example_data) == 161
assert eval_mul(example_data_b, part_b=True) == 48

puzzle.answer_a = eval_mul(data)
puzzle.answer_b = eval_mul(data, part_b=True)

# Day 4

In [41]:
puzzle = Puzzle(year=2024, day=4)

example_input = """MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX"""

example_data = create_two_d_matrix(example_input)
data = create_two_d_matrix(puzzle.input_data)

In [42]:
def check_xmas(text: str) -> int:
    """Search xmas in a string (can be reversed)"""
    pattern = {"XMAS", "SAMX"}
    return sum(text[i : i + 4] in pattern for i in range(0, len(text) - 3))


def check_x_mas(d1: str, d2: str) -> int:
    """
    Find this pattern of X-MAS (cross MAS)

    M.S
    .A.
    M.S

    the MAS can be reversed also on both diagonal
    """
    pattern = {"MAS", "SAM"}
    return int(d1 in pattern and d2 in pattern)


def number_of_xmas(text: np.array) -> int:
    dim, count = len(text), 0
    for i in range(0, dim):
        line = "".join(text[i])
        count += check_xmas(line)

        col = "".join(text[:, i])
        count += check_xmas(col)

    for offset in range(-dim, dim):
        d1 = "".join(np.diag(text, offset))
        count += check_xmas(d1)

        d2 = "".join(np.diag(np.rot90(text), offset))
        count += check_xmas(d2)

    return count


def number_of_x_mas(text: np.array) -> int:
    dim, count = len(text), 0
    for i in range(0, dim - 2):
        for j in range(0, dim - 2):
            subset = text[i : i + 3, j : j + 3]
            d1 = "".join(np.diag(subset))
            d2 = "".join(np.diag(np.rot90(subset)))
            count += check_x_mas(d1, d2)

    return count


# tests
assert number_of_xmas(example_data) == 18
assert number_of_x_mas(example_data) == 9

puzzle.answer_a = number_of_xmas(data)
puzzle.answer_b = number_of_x_mas(data)

# Day 5

In [43]:
puzzle = Puzzle(year=2024, day=5)


def parse_data(data):
    data_deps, data_updates = data.split("\n\n")

    deps = {}
    for d in data_deps.split("\n"):
        k, v = map(int, d.split("|"))
        if k in deps:
            deps[k].append(v)
        else:
            deps[k] = [v]

    updates = [list(map(int, l.split(","))) for l in data_updates.split("\n")]

    return deps, updates

In [44]:
def new_page_index(update, page_deps):
    return min(update.index(v) for v in page_deps if v in update)


def valid_update(update, deps):
    for i in range(1, len(update)):
        if any(x in update[:i] for x in deps.get(update[i], [])):
            return False
    return True


def get_correct_updates(deps, updates):
    correct_updates = []

    for update in updates:
        if valid_update(update, deps):
            correct_updates.append(update)

    return sum([l[len(l) // 2] for l in correct_updates])


def fix_incorrect_updates(deps, updates):
    incorrect_updates = []
    for update in updates:
        if not valid_update(update, deps):
            incorrect_updates.append(update)

    fixed_updates = []
    for update in incorrect_updates:
        while not valid_update(update, deps):
            for i in range(1, len(update)):
                if any(x in update[:i] for x in deps.get(update[i], [])):
                    index = new_page_index(update[:i], deps.get(update[i], []))
                    update.insert(index, update.pop(i))
                    break
        fixed_updates.append(update)

    return sum([l[len(l) // 2] for l in fixed_updates])


# parse
example_deps, example_updates = parse_data(puzzle.examples[0].input_data)
deps, updates = parse_data(puzzle.input_data)

# tests
assert get_correct_updates(example_deps, example_updates) == 143
assert fix_incorrect_updates(example_deps, example_updates) == 123

puzzle.answer_a = get_correct_updates(deps, updates)
puzzle.answer_b = fix_incorrect_updates(deps, updates)

# Day 6

In [45]:
puzzle = Puzzle(year=2024, day=6)

example_data = create_two_d_matrix(puzzle.examples[0].input_data)
data = create_two_d_matrix(puzzle.input_data)

In [46]:
def find_start(grid: np.array) -> tuple:
    i, j = np.where(grid == "^")
    return int(i[0]), int(j[0])


def max_steps(grid: np.array, pos: tuple, dir: tuple) -> int:
    """I thought this could be faster, but it's not with the current
    implementation to track the path."""
    dim = len(grid)
    if dir[1] == 0:
        i0, i1 = (0, pos[0]) if dir[0] == -1 else (pos[0], dim - 1)
        data = grid[i0 : i1 + 1, pos[1]]
    else:
        j0, j1 = (0, pos[1]) if dir[1] == -1 else (pos[1], dim - 1)
        data = grid[pos[0], j0 : j1 + 1]

    # reverse to always have the current position at index 0
    if dir[0] == -1 or dir[1] == -1:
        data = data[::-1]

    return (
        min((i for i, x in enumerate(data) if x in {"#", "O"}), default=len(data)) - 1
    )


def find_path(grid: np.array) -> str:
    dim = len(grid)
    dir = (-1, 0)
    pos = find_start(grid)  # look for "^"

    # sets to track the visited cells and identify loops
    visited = set()
    paths = set()

    # start position
    paths.add((pos, dir))

    while True:
        next_pos = (pos[0] + dir[0], pos[1] + dir[1])

        if not (0 <= next_pos[0] < dim and 0 <= next_pos[1] < dim):
            # end of the path
            break
        elif grid[next_pos] in {"#", "O"}:
            # change direction (90 deg)
            dir = (dir[1], -dir[0])
        else:
            # move
            pos = next_pos

            # add the position to the visited set
            visited.add(pos)

            # if we go back to the same position with the
            # same direction it will cause an infinite loop
            if (pos, dir) in paths:
                return None
            paths.add((pos, dir))

    return visited


def find_loop(grid: np.array) -> int:
    # extra obstacle has to be on the path
    # to possibly create a loop
    visited = find_path(grid)
    visited.remove(find_start(grid))

    count = 0
    for pos in visited:
        grid[pos] = "O"  # set an obstacle
        if not find_path(grid):
            count += 1
        grid[pos] = "."  # reset

    return count

In [47]:
assert len(find_path(example_data)) == 41
assert find_loop(example_data) == 6

puzzle.answer_a = len(find_path(data))
puzzle.answer_b = find_loop(data)  # still takes ~6s

# Day 7

In [48]:
puzzle = Puzzle(year=2024, day=7)

example_data = puzzle.examples[0].input_data
data = puzzle.input_data


got 404 status code
Please don't repeatedly request this endpoint before it unlocks! The calendar countdown is synchronized with the server time; the link will be enabled on the calendar the instant this puzzle becomes available.

unable to find example data for 2024/07 (AocdError('HTTP 404 at https://adventofcode.com/2024/day/7'))


IndexError: list index out of range

# Day 8

# Day 9

# Day 10

# Day 11

# Day 12

# Day 13

# Day 14

# Day 15

# Day 16

# Day 17

# Day 18

# Day 19

# Day 20

# Day 21

# Day 22

# Day 23

# Day 24

# Day 25